In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
train_file = '../input/facial-keypoints-detection/training.zip'
test_file = '../input/facial-keypoints-detection/test.zip'
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [2]:
train_x = train_data.Image.copy()
train_y = train_data.drop('Image', axis=1).copy()
test_x = test_data.Image.copy()

In [3]:
train_x = [[int(t) for t in i.split()] for i in train_x.to_numpy()]
train_x = np.array(train_x)
test_x = [[int(t) for t in i.split()] for i in test_x.to_numpy()]
test_x = np.array(test_x)

In [4]:
train_y.fillna(train_y.mean(), inplace=True)
train_x = train_x.reshape(train_x.shape[0], 96, 96, 1)
test_x = test_x.reshape(test_x.shape[0], 96, 96, 1)

In [5]:
train_x = train_x.astype("float32")/255.
test_x = test_x.astype("float32")/255.

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
with tpu_strategy.scope():
    model = Sequential()
    model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',
                 input_shape = (96, 96, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(strides=(2,2)))
    model.add(Dropout(0.25))


    model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(30))

Using TensorFlow backend.


In [7]:
with tpu_strategy.scope():
    model.compile(loss='mse', optimizer = Adam(lr=1e-4))

In [8]:
from keras.callbacks import LearningRateScheduler
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
train_y = train_y.to_numpy()
mean = train_y.mean(axis=0)
train_y -= mean
std  = train_y.std(axis=0)
train_y /= std

In [9]:
model.fit(train_x, train_y,
          batch_size=128,
          epochs=10,
          callbacks=[annealer],
          validation_split=0.1
         )

Train on 6344 samples, validate on 705 samples
Epoch 1/10
6344/6344 [==============================] - 26s 4ms/step - loss: 1.8552 - val_loss: 22.5518
Epoch 2/10
6344/6344 [==============================] - 23s 4ms/step - loss: 0.9740 - val_loss: 24.4139
Epoch 3/10
6344/6344 [==============================] - 23s 4ms/step - loss: 0.8765 - val_loss: 21.9914
Epoch 4/10
6344/6344 [==============================] - 23s 4ms/step - loss: 0.8256 - val_loss: 12.3880
Epoch 5/10
6344/6344 [==============================] - 23s 4ms/step - loss: 0.7770 - val_loss: 0.7707
Epoch 6/10
6344/6344 [==============================] - 22s 4ms/step - loss: 0.7328 - val_loss: 0.4027
Epoch 7/10
6344/6344 [==============================] - 23s 4ms/step - loss: 0.6970 - val_loss: 0.3056
Epoch 8/10
6344/6344 [==============================] - 23s 4ms/step - loss: 0.6670 - val_loss: 0.3057
Epoch 9/10
6344/6344 [==============================] - 23s 4ms/step - loss: 0.6389 - val_loss: 0.3203
Epoch 10/10
6344/6344 

In [10]:
res1 = model.predict(test_x)
res1 = res1*std + mean
df = pd.DataFrame(data=res1, columns=train_data.drop('Image', axis=1).copy().columns)
tablepath = '../input/facial-keypoints-detection/IdLookupTable.csv'
table = pd.read_csv(tablepath)
table['pepega'] = list(zip(table['ImageId'], table['FeatureName']))
table['Location'] = table['pepega'].map(lambda x: df.loc[x[0]-1][x[1]])
restable = table.drop(['FeatureName','pepega', 'ImageId'], axis=1)

In [11]:
restable.to_csv('../working/second.csv', index=False)